In [9]:
import json
import os
"""
DESC: Converted the original format of the batches to the new one that only includes requires rewrite
* dialogs = number of dialogs to have in a batch, -1 to include all

"""

def decreas_turn_num(json_data, offset = -1):
    for dialog_data in json_data.values():
        for turn in dialog_data['dialog']:
            turn['turn_num'] = turn['turn_num'] + offset
            
        for turn in dialog_data['annotations']:
            turn['turn_num'] = turn['turn_num'] + offset
            
    return json_data
        

def process_json(json_data, dialogs = -1):
    # Read the JSON data from the input file
    new_dialog = {}
    # Loop through the data and process as required
    counter = 0
    for dialog_key, dialog_data in json_data.items():
        if dialogs != -1:
            if counter >= dialogs:
                break
            else:
                counter += 1
            
        dialog_data['annotator_name'] = None
        annotations = []
        key_to_remove = ['annotator_id', 'number_of_turns']
        for key, value in dialog_data.items():
            rewrites = {}
            # Check if the key is a digit
            if key.isdigit() and key == "1":
                key_to_remove.append(key)
            elif key.isdigit():

                for tkey, tvalue in value.items():
                    if tkey != "annotator rewrite" and tkey != "requires rewrite":
                        rewrites[tkey] = {"text": tvalue['text'], "score": None, "optimal": None}

                # Add the 'requires_rewrite' key
                annotations.append({
                    'turn_num': int(key),
                    'turn_id': dialog_data['dialog'][int(key)-1]['turn_id'] , 
                    'requires_rewrite': None ,
                    'annotator_rewrite': None,
                    'rewrites': rewrites})
                
                key_to_remove.append(key)
        
        dialog_data['annotations'] = annotations

        for key in key_to_remove:
            del dialog_data[key]
            
        new_dialog[dialog_key] = dialog_data

    return decreas_turn_num(new_dialog)


def convert_all_json_batches(folder_path, folder_output_path, dialog = -1):
    batch_num = 1
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as file:
                data = json.load(file)
                json_data = process_json(data, dialog)
                output_path = os.path.join(folder_output_path, (f'batch_{batch_num}_src.json')) 
                output_path2 = os.path.join('target.json')
                # Write the modified data to the output file
                with open(output_path, 'w') as file:
                    json.dump(json_data, file, indent=4)
                with open(output_path2, 'w') as file:
                    json.dump(json_data, file, indent=4)
                batch_num+=1
   

# Define the input and output paths
folder_path = "older-version-program\\new-dataset\\"
folder_output_path = "new-dataset-files"



# Process the JSON data and save it
convert_all_json_batches(folder_path, folder_output_path)


In [1]:
import json
import os
def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def execute_function_on_folder(input_path, output_path, function):
    batch_num = 1
    for file_name in os.listdir(input_path):
        if file_name.endswith('.json'):
            file_path = os.path.join(input_path, file_name)
            data = read_json_file(file_path)
            new_data = function(data)
            output_file_path = os.path.join(output_path, (f'batch_{batch_num}_src.json')) 
            # Write the modified data to the output file
            save_json_file(output_file_path, new_data)
            batch_num+=1
    
def cut_dialog_data(dialog_data, turns):
    new_dialog = {}
    # Loop through the data and process as required
    counter = 0
    for key, data in dialog_data.items():
        if key == "dialog":
            counter = 0
            new_dialog[key] = []
            for turn in data:

                if counter >= turns:
                    break
                else:
                    counter += 1
                    
                new_dialog[key].append(turn)
                
        elif key == "annotations":
            counter = 1
            new_dialog[key] = []
            for turn in data:
                if counter >= turns:
                    break
                else:
                    counter += 1
                if counter != 0:
                    new_dialog[key].append(turn)

        else:
            new_dialog[key] = data

    return new_dialog

def cut_json_data(json_data, dialogs = 2, turns = 4):
    new_data = {}
    dialog_counter = 0
    for key, data in json_data.items():
        if dialog_counter >= dialogs:
            break
        new_data[key] = cut_dialog_data(data, turns)
        dialog_counter += 1
    return new_data
    
def save_json_file(output_path, data):
   with open(output_path, 'w') as file:
        json.dump(data, file, indent=4)

folder_path = "old-dataset-files"
folder_output_path = "old-dataset-files-cut"
execute_function_on_folder(folder_path, folder_output_path, cut_json_data)


In [ ]:
def find_duplicate_id(json_data):
    dialog_ids = {}
    turn_ids = {}
    for dialog_key, dialog_data in json_data.items():

        if dialog_key in dialog_ids.keys():
            print(f"Duplicate ID {dialog_key}")
            dialog_ids[dialog_key] += 1
        else:
            dialog_ids[dialog_key] = 1

        for turn in dialog_data['dialog']:

            if turn['turn_id'] in turn_ids.keys():

                print(f"Duplicate ID {turn['turn_id']} in dialog {dialog_key}")
                turn_ids[turn['turn_id']] += 1

            else:
                dialog_ids.append(turn['turn_id'])